In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are world class technical documentation writer."),
        ("user", "{input}"),
    ]
)    

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain.invoke(
    {
        "input": "What is LangGraph?"
    }
))

LangGraph is a software tool used for visualizing the relationships between programming languages. It provides a graphical representation of how different programming languages are connected based on various factors such as syntax, design principles, and historical influences. LangGraph helps developers and researchers gain insights into the similarities and differences between programming languages, as well as understand the evolution of programming language paradigms over time.


Retrieval: Chain of Models
It is used when there is data to feed the model. By showing data to the model, we can help it specialize in that particular area. A pdf, database or website can be used to feed the model with data. The model can then be used to answer questions or generate text based on the data it has been trained on.

In [5]:
# WebBaseLoader is used to retrieve data from a website. This data can then be used to train the model.
from langchain_community.document_loaders import WebBaseLoader 

loader = WebBaseLoader("https://blog.langchain.dev/langgraph/") # WebBaseLoader is used to retrieve data from a website. This data can then be used to train the model.

docs = loader.load() # This will return the data from the website.
docs 

[Document(page_content='\n\n\nLangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024\n\n\n\n\n\nTL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we wi

Docs: This will return the data from the website. 

After that We will convert the page into a digital format with embeddings. We will then store these numerical representations in a vector db. Then we will enable the model to retrieve the information in this vector database.

In [6]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()  # OpenAIEmbeddings is used to convert the page into a digital format with embeddings.

In [7]:
from langchain_community.vectorstores import FAISS
# FAISS is used to store these numerical representations in a vector db.
# We will use a database to calculate numerical representations. This database will be used to store and query numerical representations.

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# We cannot give the downloaded website data to the model as a whole. There is a certain number of tokens that the model accepts. So we need to break down the text. We will use Recursive Character Splitter for this splitting process.

text_splitter = RecursiveCharacterTextSplitter() # Recursive Character Splitter is used to break down the text.
documents = text_splitter.split_documents(docs) # This will return the split text.
documents

[Document(page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024', metadata={'source': 'http://blog.langchain.dev/langgraph/', 'title': 'LangGraph', 'language': 'en'}),
 Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an

Documents: This will return the split text. So we have split the text into smaller parts. Now we will convert these parts into embeddings.

In [10]:
vector = FAISS.from_documents(documents, embeddings) # This will return the numerical representations of the documents. So we have converted the text into numerical representations. And stored these numerical representations in a vector database.

The html page is now indexed in the vector database. We can now use the model to retrieve information from this vector database.

Now we will create a retrieval chain. This chain will receive the incoming question, then look at the relevant document and then transfer these documents to a model with the original question. This model will produce answers according to these documents.

In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
    Answer the following question based only on the provided context:
    <context>
    {context}
    </context>
    Question: {input}
    """
)
# Context is the data that the model will use to answer the question. The model will only use this data to answer the question.

We created the template. Now we will create a chain to pass the documents to the template.

In [12]:
from langchain.chains.combine_documents import create_stuff_documents_chain
# create_stuff_documents_chain is used to create a chain to pass the documents to the template.

document_chain = create_stuff_documents_chain(model, prompt)

Now we will create a retrieval chain to capture the document related to the question asked.

In [13]:
from langchain.chains import create_retrieval_chain
# create_retrieval_chain combines the documents in vector db with the template created in document_chain.

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001ACE47837C0>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\n    Answer the following question based only on the provided context:\n    <context>\n    {context}\n    </context>\n    Question: {input}\n    '))])
            | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001ACE111E3E0>, async_client=<openai.resour

We created the chain that brings information about the question asked. From now on, the model will answer the questions by looking at the website.

In [14]:
response = retrieval_chain.invoke(
    {
        "input": "What is LangGraph?"
    }
)
print(response)

{'input': 'What is LangGraph?', 'context': [Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we\'ve implemented already. We will then highlight a few of the common modifications to these runtimes we\'ve heard requests for, and examples of implementing those. We will finish with a preview of what we will b

In [15]:
print(response["answer"])

LangGraph is a module built on top of LangChain that enables the creation of cyclical graphs, which are often needed for agent runtimes. It adds new value by providing an easy way to introduce cycles into chains, allowing for more complex and flexible applications to be developed.


The model has answered the question by looking at the website. It has answered the question by looking at the data on the website.
This is how we can create a retrieval chain to answer questions by looking at the data on the website.

If we want, we can get an answer by just giving a part of the document. Now let's do that.

In [18]:
from langchain_core.documents import Document


doc = Document(
    page_content="LangGraph is a library for building stateful, multi-actor applications with LLMs. Inspired by Pregel and Apache Beam, LangGraph lets you coordinate and checkpoint multiple chains (or actors) across cyclic computational steps using regular python functions (or JS). The public interface draws inspiration from NetworkX."
)
# We have created a document with a part of the website data. Now we will use this document to answer the question.

In [19]:
document_chain.invoke(
    {
        "input": "What is LangGraph?",
        "context": [doc]
    }
)

'LangGraph is a library for building stateful, multi-actor applications with LLMs.'

Retrival chain is used to retrieve data from a website. This data can then be used to train the model. The model can then be used to answer questions or generate text based on the data it has been trained on. We can also use a part of the document to answer the question. So we can get an answer by just giving a part of the document. 